# Challenge Part 2
## Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
%matplotlib inline

# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
import scipy.stats as sts
# Import mpl to change the plot configurations using rcParams.
import matplotlib as mpl

import random 
import timeit
import requests
import time
import gmaps

#Notice the difference in import syntax for these modules. This eliminates the need to double type the name
#For example: datetime.datetime.xyz
from citipy import citipy
from datetime import datetime
from scipy.stats import linregress

# Import API keys.
from config import weather_api_key

from config import g_key
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Part 2 section 2 ####################################################################################
# Load in csv 
city_weatherDB_to_load = os.path.join("weather_data", "WeatherPy_database.csv")

city_weather_df = pd.read_csv(city_weatherDB_to_load)

In [2]:
city_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain last hr,Snow last hr
0,0,Cruz Alta,-28.64,-53.61,45.05,85,58,12.84,BR,2020-06-27 23:25:44,descriptionexception,0.00,0.0
1,1,Hobart,-42.88,147.33,39.99,93,20,9.17,AU,2020-06-27 23:25:44,descriptionexception,0.00,0.0
2,2,Tuatapere,-46.13,167.68,47.59,85,100,3.87,NZ,2020-06-27 23:25:44,descriptionexception,0.00,0.0
3,3,Lebu,-37.62,-73.65,51.80,88,100,12.59,CL,2020-06-27 23:25:44,descriptionexception,0.69,0.0
4,4,Tasiilaq,65.61,-37.64,42.80,87,100,1.12,GL,2020-06-27 23:25:44,descriptionexception,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,566,Paamiut,61.99,-49.67,41.34,84,88,1.79,GL,2020-06-27 23:27:15,descriptionexception,0.00,0.0
567,567,Hirara,24.80,125.28,86.00,94,75,8.05,JP,2020-06-27 23:27:15,descriptionexception,0.00,0.0
568,568,Chiang Klang,19.29,100.86,77.00,100,40,1.12,TH,2020-06-27 23:27:15,descriptionexception,0.00,0.0
569,569,Puerto Lempira,15.27,-83.77,77.00,88,90,6.98,HN,2020-06-27 23:27:15,descriptionexception,0.00,0.0


In [3]:
city_weather_df.drop(city_weather_df.columns[[0]], axis=1, inplace=True)

In [4]:
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain last hr,Snow last hr
0,Cruz Alta,-28.64,-53.61,45.05,85,58,12.84,BR,2020-06-27 23:25:44,descriptionexception,0.00,0.0
1,Hobart,-42.88,147.33,39.99,93,20,9.17,AU,2020-06-27 23:25:44,descriptionexception,0.00,0.0
2,Tuatapere,-46.13,167.68,47.59,85,100,3.87,NZ,2020-06-27 23:25:44,descriptionexception,0.00,0.0
3,Lebu,-37.62,-73.65,51.80,88,100,12.59,CL,2020-06-27 23:25:44,descriptionexception,0.69,0.0
4,Tasiilaq,65.61,-37.64,42.80,87,100,1.12,GL,2020-06-27 23:25:44,descriptionexception,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
566,Paamiut,61.99,-49.67,41.34,84,88,1.79,GL,2020-06-27 23:27:15,descriptionexception,0.00,0.0
567,Hirara,24.80,125.28,86.00,94,75,8.05,JP,2020-06-27 23:27:15,descriptionexception,0.00,0.0
568,Chiang Klang,19.29,100.86,77.00,100,40,1.12,TH,2020-06-27 23:27:15,descriptionexception,0.00,0.0
569,Puerto Lempira,15.27,-83.77,77.00,88,90,6.98,HN,2020-06-27 23:27:15,descriptionexception,0.00,0.0


In [5]:
# Part 2 section 3 ####################################################################################
# Ask the customer prompts
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


rain_toggle = str(input("Do you want it to be raining? (yes/no) "))
snow_toggle = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
#Filter by temperature range
preferred_cities_df = city_weather_df.loc[(city_weather_df["Max Temp"] <= max_temp) & \
                                       (city_weather_df["Max Temp"] >= min_temp)]

preferred_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain last hr,Snow last hr
8,Ixtapa,20.70,-105.20,87.80,66,75,11.41,MX,2020-06-27 23:25:44,descriptionexception,0.00,0.0
15,Jamestown,42.10,-79.24,75.00,73,75,14.99,US,2020-06-27 23:24:09,descriptionexception,0.00,0.0
17,San Cristobal,7.77,-72.22,87.80,58,40,8.05,VE,2020-06-27 23:22:12,descriptionexception,0.00,0.0
24,Kapaa,22.08,-159.32,84.99,66,1,14.99,US,2020-06-27 23:25:13,descriptionexception,0.00,0.0
27,Faanui,-16.48,-151.75,76.93,74,60,15.50,PF,2020-06-27 23:25:45,descriptionexception,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
562,Devils Lake,48.11,-98.87,89.60,38,1,17.22,US,2020-06-27 23:26:36,descriptionexception,0.00,0.0
563,Ternate,0.80,127.40,80.08,78,81,1.70,ID,2020-06-27 23:27:15,descriptionexception,0.35,0.0
567,Hirara,24.80,125.28,86.00,94,75,8.05,JP,2020-06-27 23:27:15,descriptionexception,0.00,0.0
568,Chiang Klang,19.29,100.86,77.00,100,40,1.12,TH,2020-06-27 23:27:15,descriptionexception,0.00,0.0


In [7]:
#Filter by RAIN (filtering on the prefered cities generated byt he temerature filter)
if rain_toggle == "no":
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain last hr"] == 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain last hr"] != 0)]

preferred_cities_df     

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain last hr,Snow last hr
8,Ixtapa,20.70,-105.20,87.80,66,75,11.41,MX,2020-06-27 23:25:44,descriptionexception,0.0,0.0
15,Jamestown,42.10,-79.24,75.00,73,75,14.99,US,2020-06-27 23:24:09,descriptionexception,0.0,0.0
17,San Cristobal,7.77,-72.22,87.80,58,40,8.05,VE,2020-06-27 23:22:12,descriptionexception,0.0,0.0
24,Kapaa,22.08,-159.32,84.99,66,1,14.99,US,2020-06-27 23:25:13,descriptionexception,0.0,0.0
27,Faanui,-16.48,-151.75,76.93,74,60,15.50,PF,2020-06-27 23:25:45,descriptionexception,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
555,Karari,25.45,81.43,84.99,59,98,5.48,IN,2020-06-27 23:27:14,descriptionexception,0.0,0.0
562,Devils Lake,48.11,-98.87,89.60,38,1,17.22,US,2020-06-27 23:26:36,descriptionexception,0.0,0.0
567,Hirara,24.80,125.28,86.00,94,75,8.05,JP,2020-06-27 23:27:15,descriptionexception,0.0,0.0
568,Chiang Klang,19.29,100.86,77.00,100,40,1.12,TH,2020-06-27 23:27:15,descriptionexception,0.0,0.0


In [8]:
#Filter by SNOW (filtering on the prefered cities generated by the temerature & rain filters)
if snow_toggle == "no":
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow last hr"] == 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow last hr"] != 0)]

preferred_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description,Rain last hr,Snow last hr
8,Ixtapa,20.70,-105.20,87.80,66,75,11.41,MX,2020-06-27 23:25:44,descriptionexception,0.0,0.0
15,Jamestown,42.10,-79.24,75.00,73,75,14.99,US,2020-06-27 23:24:09,descriptionexception,0.0,0.0
17,San Cristobal,7.77,-72.22,87.80,58,40,8.05,VE,2020-06-27 23:22:12,descriptionexception,0.0,0.0
24,Kapaa,22.08,-159.32,84.99,66,1,14.99,US,2020-06-27 23:25:13,descriptionexception,0.0,0.0
27,Faanui,-16.48,-151.75,76.93,74,60,15.50,PF,2020-06-27 23:25:45,descriptionexception,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
555,Karari,25.45,81.43,84.99,59,98,5.48,IN,2020-06-27 23:27:14,descriptionexception,0.0,0.0
562,Devils Lake,48.11,-98.87,89.60,38,1,17.22,US,2020-06-27 23:26:36,descriptionexception,0.0,0.0
567,Hirara,24.80,125.28,86.00,94,75,8.05,JP,2020-06-27 23:27:15,descriptionexception,0.0,0.0
568,Chiang Klang,19.29,100.86,77.00,100,40,1.12,TH,2020-06-27 23:27:15,descriptionexception,0.0,0.0


In [9]:
preferred_cities_df.shape

(146, 12)

In [10]:
# Part 2 sections 4,5,6 ####################################################################################

In [17]:
#Generate hotel dataframe - hotel column will populate through a later API call
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
8,Ixtapa,MX,87.80,20.70,-105.20,descriptionexception,
15,Jamestown,US,75.00,42.10,-79.24,descriptionexception,
17,San Cristobal,VE,87.80,7.77,-72.22,descriptionexception,
24,Kapaa,US,84.99,22.08,-159.32,descriptionexception,
27,Faanui,PF,76.93,-16.48,-151.75,descriptionexception,
28,Manavalakurichi,IN,80.60,8.13,77.30,descriptionexception,
33,Avarua,CK,75.20,-21.21,-159.78,descriptionexception,
34,Lazaro Cardenas,MX,86.43,17.96,-102.20,descriptionexception,
36,Sosua,DO,84.20,19.77,-70.52,descriptionexception,
42,Dongying,CN,80.53,37.46,118.49,descriptionexception,


In [18]:
hotel_df.shape

(146, 7)

In [19]:
# Set API parameters to search for a hotel.

# 5000 is in meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [20]:
# Iterate through the DataFrame for each lat & lon
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Ixtapa,MX,87.80,20.70,-105.20,Container Inn Puerto Vallarta
15,Jamestown,US,75.00,42.10,-79.24,Hampton Inn & Suites Jamestown
17,San Cristobal,VE,87.80,7.77,-72.22,Pirineos
24,Kapaa,US,84.99,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
27,Faanui,PF,76.93,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
...,...,...,...,...,...,...
555,Karari,IN,84.99,25.45,81.43,Pushp Vatika guest house
562,Devils Lake,US,89.60,48.11,-98.87,Holiday Inn Express Devils Lake
567,Hirara,JP,86.00,24.80,125.28,Hotel Atoll Emerald Miyakojima
568,Chiang Klang,TH,77.00,19.29,100.86,Sangthongresort


In [16]:
hotel_df.count()

City          146
Country       146
Max Temp      146
Lat           146
Lng           146
Hotel Name    146
dtype: int64

In [22]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
#Edited code with map markers
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# The pop-up box uses html formatting
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [24]:
# Updated map with pop-up added
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))